In [12]:
import os
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Global settings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT).to(device)
resnet.eval()  # Set to evaluation mode
feature_extractor = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove classification head


def preprocess_dataset(dataset_path):
    """
    Load and preprocess the dataset using ImageFolder.
    """
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
    ])
    dataset = ImageFolder(dataset_path, transform=transform)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False)
    return dataloader, dataset.classes


def extract_features(dataloader, model):
    """
    Extract features using the pre-trained ResNet model.
    """
    features, labels = [], []
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            outputs = outputs.view(outputs.size(0), -1)  # Flatten features
            features.append(outputs.cpu().numpy())
            labels.append(targets.numpy())
    return np.vstack(features), np.hstack(labels)


def train_svm(features, labels):
    """
    Train an SVM classifier using the extracted features.
    """
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    svm = SVC(kernel='linear', random_state=42)
    svm.fit(features_scaled, labels)
    return svm, scaler


def evaluate_model(svm, scaler, features, labels, class_names, dataset_name):
    """
    Evaluate the SVM model and save the confusion matrix.
    """
    features_scaled = scaler.transform(features)
    predictions = svm.predict(features_scaled)

    # Generate classification report
    print(f"Classification Report for {dataset_name}:\n")
    print(classification_report(labels, predictions, target_names=class_names))

    # Generate confusion matrix
    cm = confusion_matrix(labels, predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap="Blues")
    plt.title(f"Confusion Matrix for {dataset_name}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()

    # Create output folder if it doesn't exist
    output_folder = "output_cnn"
    os.makedirs(output_folder, exist_ok=True)

    # Save confusion matrix as image
    output_path = os.path.join(output_folder, f"{dataset_name}_confusion_matrix.png")
    plt.savefig(output_path)
    plt.close()
    print(f"Confusion matrix saved as {output_path}")


def process_dataset(dataset_path):
    """
    Process a single dataset: load, extract features, train SVM, evaluate and save confusion matrix.
    """
    dataset_name = os.path.basename(dataset_path)
    print(f"\nProcessing dataset: {dataset_name}")

    dataloader, class_names = preprocess_dataset(dataset_path)
    features, labels = extract_features(dataloader, feature_extractor)
    svm, scaler = train_svm(features, labels)
    evaluate_model(svm, scaler, features, labels, class_names, dataset_name)


# Main script
dataset_paths = ["Dataset1", "Dataset2", "Dataset3"]

for dataset_path in dataset_paths:
    process_dataset(dataset_path)



Processing dataset: Dataset1
Classification Report for Dataset1:

              precision    recall  f1-score   support

     Laptops       1.00      1.00      1.00        20
 Smartphones       1.00      1.00      1.00        20

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40

Confusion matrix saved as output_cnn\Dataset1_confusion_matrix.png

Processing dataset: Dataset2
Classification Report for Dataset2:

              precision    recall  f1-score   support

        Cars       1.00      1.00      1.00        20
      Planes       1.00      1.00      1.00        20

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40

Confusion matrix saved as output_cnn\Dataset2_confusion_matrix.png

Processing dataset: Dataset3
Classification Report for Dataset3:

           